# Codebusters KPIs

## import libraries

In [79]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob

Import and clean `all_tickets_2022` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all service request from partners

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (SR)") AND labels in (Partners) AND resolved is not EMPTY AND resolutiondate >= 2022-12-19`

In [80]:
all_partners_origin = pd.read_csv('../../DataSets/MainTickets/all_partners.csv')
all_partners_origin

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Custom field (iPPM Activity),Custom field (iPPM Item Description),Custom field (iPPM SO),Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,Numiga VFL Wolfsburg - Hidde accommodation sec...,CYTCI-10248122,7817266,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Numiga Fiege IT - Edit system texts,CYTCI-10248121,7816347,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,2023-10-12 11:20;pklosinski;Hi [~alejandro.rui...,2023-10-13 14:06;alejandro.ruizrobles;Hi [~pkl...,2023-10-17 07:08;pklosinski;I have to add brea...,2023-10-17 11:59;pklosinski;Hi [~alejandro.rui...,NaN,NaN,NaN
2,Numiga Rhenus - Small change in a FDF,CYTCI-9607,8018078,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Numiga VFL Wolfsburg - Remove a configuration,CYTCI-9546,7979253,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,"2023-11-15 15:52;jbaraniecka;Hi, [~alejandro.r...",2023-11-15 16:48;alejandro.ruizrobles;Hi [~jba...,"2023-11-20 10:40;jbaraniecka;HI, [~alejandro.r...",NaN,NaN,NaN,NaN
4,Derpart - Create user credentials for certifie...,CYTCI-9495,7927668,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Numiga Elli - Change SFTP path for export file,CYTCI-6523,7651954,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,Numiga Duravit - Create a new system,CYTCI-6500,7651931,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,2022-03-09 09:50;alejandro.ruizrobles;Hi [~mku...,2022-03-09 17:47;alejandro.ruizrobles;Hi [~mku...,2022-03-10 10:45;mstaszek;Hello [~aruizrobles]...,NaN,NaN,NaN,NaN
192,Numiga Eberhard AG - Change FDFs,CYTCI-6499,7651930,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Numiga Elli - Remove receipt,CYTCI-6495,7651926,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Utilites functions

In [81]:
def find_escalatetions(row, dataSet):
    label_columns = [col for col in dataSet if 'label' in col.lower()]
    isEscalated = False
    for column in label_columns:
        if 'escalated' in str(row[column]).lower():
            isEscalated = True    
    return isEscalated

# Analyze labels

## Find any escalated ticekt

In [82]:
all_partners_escalated_match = all_partners_origin
all_partners_escalated_match["Is Escalated"] = all_partners_escalated_match.apply(lambda row: find_escalatetions(row, all_partners_escalated_match), axis=1)
all_partners_escalated_match

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Custom field (iPPM Item Description),Custom field (iPPM SO),Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Is Escalated
0,Numiga VFL Wolfsburg - Hidde accommodation sec...,CYTCI-10248122,7817266,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,Numiga Fiege IT - Edit system texts,CYTCI-10248121,7816347,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,2023-10-12 11:20;pklosinski;Hi [~alejandro.rui...,2023-10-13 14:06;alejandro.ruizrobles;Hi [~pkl...,2023-10-17 07:08;pklosinski;I have to add brea...,2023-10-17 11:59;pklosinski;Hi [~alejandro.rui...,NaN,NaN,NaN,False
2,Numiga Rhenus - Small change in a FDF,CYTCI-9607,8018078,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Numiga VFL Wolfsburg - Remove a configuration,CYTCI-9546,7979253,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,"2023-11-15 15:52;jbaraniecka;Hi, [~alejandro.r...",2023-11-15 16:48;alejandro.ruizrobles;Hi [~jba...,"2023-11-20 10:40;jbaraniecka;HI, [~alejandro.r...",NaN,NaN,NaN,NaN,False
4,Derpart - Create user credentials for certifie...,CYTCI-9495,7927668,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Numiga Elli - Change SFTP path for export file,CYTCI-6523,7651954,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
191,Numiga Duravit - Create a new system,CYTCI-6500,7651931,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,2022-03-09 09:50;alejandro.ruizrobles;Hi [~mku...,2022-03-09 17:47;alejandro.ruizrobles;Hi [~mku...,2022-03-10 10:45;mstaszek;Hello [~aruizrobles]...,NaN,NaN,NaN,NaN,False
192,Numiga Eberhard AG - Change FDFs,CYTCI-6499,7651930,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
193,Numiga Elli - Remove receipt,CYTCI-6495,7651926,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


# Filter the data

### Select from the dataset only columns necessary for the further analysis

In [83]:
all_partners = all_partners_escalated_match[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)', 'Is Escalated']]
all_partners.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\2459505945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [84]:
all_partners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        195 non-null    object
 1   Issue id         195 non-null    int64 
 2   Status           195 non-null    object
 3   Reporter         195 non-null    object
 4   Created          195 non-null    object
 5   Due Date         195 non-null    object
 6   Resolved         195 non-null    object
 7   Customer         195 non-null    object
 8   Type of Request  195 non-null    object
 9   Is Escalated     195 non-null    bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 14.0+ KB


In [85]:
all_partners.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
dtype: int64

### Convert date table to `datetime` object

In [86]:
all_partners["Created"] = pd.to_datetime(all_partners["Created"])
all_partners["Resolved"] = pd.to_datetime(all_partners["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\3266364689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners["Created"] = pd.to_datetime(all_partners["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\3266364689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners["Resolved"] = pd.to_datetime(all_partners["Resolved"])


In [87]:
all_partners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        195 non-null    object        
 1   Issue id         195 non-null    int64         
 2   Status           195 non-null    object        
 3   Reporter         195 non-null    object        
 4   Created          195 non-null    datetime64[ns]
 5   Due Date         195 non-null    object        
 6   Resolved         195 non-null    datetime64[ns]
 7   Customer         195 non-null    object        
 8   Type of Request  195 non-null    object        
 9   Is Escalated     195 non-null    bool          
dtypes: bool(1), datetime64[ns](2), int64(1), object(6)
memory usage: 14.0+ KB


In [88]:
all_partners.shape

(195, 10)

In [89]:
all_partners["ticket_age"] = all_partners["Resolved"] - all_partners["Created"]
all_partners

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\3972566984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners["ticket_age"] = all_partners["Resolved"] - all_partners["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00
...,...,...,...,...,...,...,...,...,...,...,...
190,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00
191,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00
192,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00
193,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00


**Summary**
- there is not partners ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

# Prepare data with individuals tasks for service requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [90]:
all_individual_tasks_path = glob.glob("../../DataSets/IndividualTasks/" + "*.csv")


all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (111,196,261,431,489) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (21,26,39,40,41,80,83,94,267,296,306,438,439,497) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (35,36,109,112,262,430,431,432,433,434,435,436,437,438,439,440,441,442,486,500,548,549) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individ

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (25,26,402,403,404,405,406,407,408,409,410,411,412,413,463) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (26,36,37,38,111,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,550,551,552,553) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\986537800.py:4: DtypeWarning: Columns (25,27,28,29,38,39,40,41,

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [91]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33845,1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33846,1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [93]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Review,7679692,7650050,2021-02-27 14:46,2021-02-27 14:46
1,Clarify,7679691,7650034,2021-02-26 17:29,2021-02-26 17:29
2,Explain,7679689,7650029,2021-02-26 15:37,2021-02-26 15:37
3,Communicate,7679687,7649877,2021-02-26 15:19,2021-02-26 15:20
4,Import in Production,7679686,7649877,2021-02-26 15:19,2021-02-26 15:19
...,...,...,...,...,...
33845,Configure,7705909,7654080,2023-05-29 12:27,2023-11-17 12:15
33846,Configure,7705523,7654026,2023-05-15 12:39,2023-11-17 13:04
33847,Do,7705402,7654004,2023-05-11 10:58,2023-11-17 12:30
33848,Review,7703060,7653656,2023-03-03 12:53,2023-11-02 08:56


In [94]:
all_individual_tasks[all_individual_tasks['Parent id'] == 465210]

,Summary,Issue id,Parent id,Created,Resolved


### Convert date like columns to `datetime` object

In [95]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33850 entries, 0 to 33849
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    33850 non-null  object        
 1   Issue id   33850 non-null  int64         
 2   Parent id  33850 non-null  int64         
 3   Created    33850 non-null  datetime64[ns]
 4   Resolved   33822 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 1.3+ MB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [96]:
all_individual_tasks["Created"].iloc[3766]

Timestamp('2021-02-17 16:35:00')

In [97]:
all_individual_tasks["Created"].iloc[3765]

Timestamp('2021-02-18 13:07:00')

In [98]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                                                                                   6148
Review                                                                                      5907
Communicate                                                                                 4970
Verify                                                                                      4893
Import to Test System                                                                       4405
Import in Production                                                                        4041
Import to Customer Test System                                                              1189
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      406
Do                    

### Check if there are any `null` values

In [99]:
all_individual_tasks.isnull().sum()

Summary       0
Issue id      0
Parent id     0
Created       0
Resolved     28
dtype: int64

### Drop all rows with null value

In [100]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

### Identify and drop duplicated rows

In [101]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved
32582,Review,7828796,7828795,2023-10-05 16:21:00,2023-11-02 14:35:00
33585,Communicate,7929412,7917819,2023-11-02 19:45:00,2023-11-02 19:45:00
33590,Import in Production,7928462,7877598,2023-11-02 15:58:00,2023-11-02 16:02:00
33591,Configure,7928460,7877598,2023-11-02 15:58:00,2023-11-02 15:58:00
33593,Communicate,7928316,7891707,2023-11-02 15:26:00,2023-11-02 15:37:00
...,...,...,...,...,...
33750,Configure,7892900,7811905,2023-10-23 16:34:00,2023-11-02 08:43:00
33763,Configure,7882479,7873215,2023-10-19 14:49:00,2023-11-02 13:55:00
33767,Review,7877599,7877598,2023-10-19 02:05:00,2023-11-02 15:58:00
33776,Communicate,7868731,7866065,2023-10-17 09:06:00,2023-11-02 10:49:00


In [102]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates

,Summary,Issue id,Parent id,Created,Resolved
0,Review,7679692,7650050,2021-02-27 14:46:00,2021-02-27 14:46:00
1,Clarify,7679691,7650034,2021-02-26 17:29:00,2021-02-26 17:29:00
2,Explain,7679689,7650029,2021-02-26 15:37:00,2021-02-26 15:37:00
3,Communicate,7679687,7649877,2021-02-26 15:19:00,2021-02-26 15:20:00
4,Import in Production,7679686,7649877,2021-02-26 15:19:00,2021-02-26 15:19:00
...,...,...,...,...,...
33844,Review,7706263,7654140,2023-06-07 10:44:00,2023-11-17 12:29:00
33845,Configure,7705909,7654080,2023-05-29 12:27:00,2023-11-17 12:15:00
33846,Configure,7705523,7654026,2023-05-15 12:39:00,2023-11-17 13:04:00
33847,Do,7705402,7654004,2023-05-11 10:58:00,2023-11-17 12:30:00


**Summary:**

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [103]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\1582069957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]


### List all available statuses of the CI ticket subtask

In [104]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                                                                                   6134
Review                                                                                      5873
Communicate                                                                                 4958
Verify                                                                                      4878
Import to Test System                                                                       4398
Import in Production                                                                        4037
Import to Customer Test System                                                              1179
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      405
Do                    

## Get the lates `communicate` subtaks date for the particular `Parent id`

In [105]:
all_individual_tasks_communicate_time = all_individual_tasks_time_calculation
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time[all_individual_tasks_communicate_time['Summary'] == "Communicate"]
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.sort_values(["Parent id", "Created"], ascending=[True, False])
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.drop_duplicates(subset="Parent id", keep='first')
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.filter(items=["Parent id","Created"]).set_index('Parent id')
all_individual_tasks_communicate_time.rename({"Created": "Resolved-communicate"},axis=1, inplace=True)

all_individual_tasks_communicate_time

,Resolved-communicate
Parent id,
335272,2022-10-03 11:14:00
360791,2022-10-07 15:31:00
374097,2022-11-24 16:28:00
376147,2022-10-10 14:19:00
388122,2022-10-20 17:08:00
...,...
8046096,2023-11-28 12:36:00
8048208,2023-11-28 09:47:00
8049403,2023-11-29 11:29:00


## Calculate time ticket was in review step

In [106]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,0,Review,7679692,7650050,2021-02-27 14:46:00,2021-02-27 14:46:00,0 days 00:00:00
1,16,Review,7679672,7650047,2021-02-26 14:28:00,2021-02-26 14:28:00,0 days 00:00:00
2,22,Review,7679664,7650045,2021-02-26 12:46:00,2021-02-26 13:17:00,0 days 00:31:00
3,33,Review,7679647,7650040,2021-02-25 22:41:00,2021-02-25 22:42:00,0 days 00:01:00
4,41,Review,7679633,7649967,2021-02-25 14:22:00,2021-02-25 16:31:00,0 days 02:09:00
...,...,...,...,...,...,...,...
5868,33838,Review,7706389,7654151,2023-06-12 13:30:00,2023-11-17 13:27:00,157 days 23:57:00
5869,33839,Review,7706388,7654150,2023-06-12 13:27:00,2023-11-17 13:25:00,157 days 23:58:00
5870,33840,Review,7706387,7654149,2023-06-12 13:26:00,2023-11-17 13:26:00,158 days 00:00:00
5871,33843,Review,7706264,7654141,2023-06-07 10:49:00,2023-11-17 12:28:00,163 days 01:39:00


In [107]:
all_partners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        195 non-null    object         
 1   Issue id         195 non-null    int64          
 2   Status           195 non-null    object         
 3   Reporter         195 non-null    object         
 4   Created          195 non-null    datetime64[ns] 
 5   Due Date         195 non-null    object         
 6   Resolved         195 non-null    datetime64[ns] 
 7   Customer         195 non-null    object         
 8   Type of Request  195 non-null    object         
 9   Is Escalated     195 non-null    bool           
 10  ticket_age       195 non-null    timedelta64[ns]
dtypes: bool(1), datetime64[ns](2), int64(1), object(6), timedelta64[ns](1)
memory usage: 15.5+ KB


### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [108]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 465210]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [109]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,393585,3 days 13:22:00
2,399228,52 days 06:54:00
3,402477,0 days 18:56:00
4,404450,11 days 19:36:00
...,...,...
5102,8055536,0 days 00:01:00
5103,8057074,0 days 16:43:00
5104,8060101,0 days 00:03:00
5105,8061206,0 days 00:13:00


In [110]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


In [111]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5107 entries, 0 to 5106
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       5107 non-null   int64          
 1   time_in_review  5107 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 79.9 KB


In [112]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


### Check on the particular ticket if the total time is summed correctly

In [113]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
404450,1,1,1,1,1,1
...,...,...,...,...,...,...
8055536,1,1,1,1,1,1
8057074,1,1,1,1,1,1
8060101,1,1,1,1,1,1


In [114]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [115]:
type(all_individual_tasks_reivew_sum)

pandas.core.frame.DataFrame

In [116]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


In [117]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [118]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [119]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_partners` dataset with `all_individual_tasks_reivew_sum_df`

In [120]:
all_partners_review_time = pd.merge(all_partners, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_partners_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_in_review
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266.0,3 days 02:30:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347.0,8 days 22:53:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078.0,5 days 20:40:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253.0,1 days 06:24:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668.0,0 days 20:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954.0,4 days 05:14:00
191,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931.0,10 days 01:50:00
192,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930.0,3 days 11:15:00
193,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926.0,3 days 12:32:00


In [121]:
all_partners_review_time.isnull().sum()

Issue key           0
Issue id            0
Status              0
Reporter            0
Created             0
Due Date            0
Resolved            0
Customer            0
Type of Request     0
Is Escalated        0
ticket_age          0
Parent id          32
time_in_review     32
dtype: int64

### Investigate `null` rows after merging  main partners tickets with individual subtasks

In [122]:
null_rows = all_partners_review_time[all_partners_review_time["time_in_review"].isnull()]
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 108 to 140
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        32 non-null     object         
 1   Issue id         32 non-null     int64          
 2   Status           32 non-null     object         
 3   Reporter         32 non-null     object         
 4   Created          32 non-null     datetime64[ns] 
 5   Due Date         32 non-null     object         
 6   Resolved         32 non-null     datetime64[ns] 
 7   Customer         32 non-null     object         
 8   Type of Request  32 non-null     object         
 9   Is Escalated     32 non-null     bool           
 10  ticket_age       32 non-null     timedelta64[ns]
 11  Parent id        0 non-null      float64        
 12  time_in_review   0 non-null      timedelta64[ns]
dtypes: bool(1), datetime64[ns](2), float64(1), int64(1), object(6), timedelta64[ns](2)
m

In [123]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_in_review
108,CYTCI-7876,7653281,Closed,alejandro.ruizrobles,2022-12-15 17:01:00,2022-12-21 00:00,2022-12-23 13:18:00,Numiga,Service Request (SR),False,7 days 20:17:00,NaN,NaT
109,CYTCI-7855,7653260,Closed,alejandro.ruizrobles,2022-12-12 16:54:00,2022-12-15 00:00,2022-12-13 13:52:00,Numiga,Service Request (SR),False,0 days 20:58:00,NaN,NaT
111,CYTCI-7806,7653219,Closed,alejandro.ruizrobles,2022-12-02 16:36:00,2022-12-11 00:00,2022-12-20 10:20:00,Numiga,Service Request (SR),False,17 days 17:44:00,NaN,NaT
112,CYTCI-7796,7653211,Closed,alejandro.ruizrobles,2022-12-01 18:44:00,2022-12-07 00:00,2022-12-12 15:52:00,Numiga,Service Request (SR),False,10 days 21:08:00,NaN,NaT
113,CYTCI-7774,7653189,Closed,alejandro.ruizrobles,2022-11-28 17:19:00,2022-12-13 00:00,2022-12-15 16:36:00,Lufthansa City Center,Service Request (SR),False,16 days 23:17:00,NaN,NaT
114,CYTCI-7773,7653188,Closed,alejandro.ruizrobles,2022-11-28 16:58:00,2022-12-31 00:00,2022-12-20 14:24:00,Numiga,Service Request (SR),False,21 days 21:26:00,NaN,NaT
115,CYTCI-7772,7653187,Closed,alejandro.ruizrobles,2022-11-28 16:24:00,2022-12-12 00:00,2022-12-13 16:59:00,Lufthansa City Center,Service Request (SR),False,15 days 00:35:00,NaN,NaT
116,CYTCI-7756,7653171,Closed,alejandro.ruizrobles,2022-11-23 17:19:00,2022-11-29 00:00,2022-12-12 14:08:00,Numiga,Service Request (SR),False,18 days 20:49:00,NaN,NaT
117,CYTCI-7755,7653170,Closed,alejandro.ruizrobles,2022-11-23 16:38:00,2022-11-29 00:00,2023-01-04 11:46:00,Numiga,Service Request (SR),False,41 days 19:08:00,NaN,NaT
118,CYTCI-7746,7653161,Closed,alejandro.ruizrobles,2022-11-22 11:10:00,2022-11-21 00:00,2022-11-24 17:18:00,Amadeus Pre Sales Demos,Service Request (SR),False,2 days 06:08:00,NaN,NaT


In [124]:
all_partners_review_time_without_null = all_partners_review_time.dropna()
all_partners_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

#### TO DO
- update the lates dateset

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [125]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [126]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
360791,1,1,1,1,1
376147,2,2,2,2,2
388122,1,1,1,1,1
392021,1,1,1,1,1
...,...,...,...,...,...
8055536,3,3,3,3,3
8057074,1,1,1,1,1
8060101,2,2,2,2,2


In [127]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [128]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,360791,0 days 00:01:00
2,376147,0 days 00:00:00
3,388122,28 days 16:13:00
4,392021,232 days 01:49:00
...,...,...
5337,8055536,0 days 00:06:00
5338,8057074,0 days 16:43:00
5339,8060101,0 days 00:04:00
5340,8061206,0 days 01:14:00


In [129]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [130]:
all_partners_codebusters_time = pd.merge(all_partners, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_partners_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266,6 days 03:05:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347,12 days 20:14:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078,6 days 17:43:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253,6 days 00:26:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668,0 days 21:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954,4 days 06:42:00
160,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931,12 days 13:06:00
161,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930,4 days 10:25:00
162,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926,4 days 11:46:00


### Investigate `null` rows after merging  main partners tickets with individual subtasks

In [131]:
all_partners_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [132]:
all_partners_codebusters_time_without_null = all_partners_codebusters_time.dropna()
all_partners_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [133]:
all_partners_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266,6 days 03:05:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347,12 days 20:14:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078,6 days 17:43:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253,6 days 00:26:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668,0 days 21:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954,4 days 06:42:00
160,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931,12 days 13:06:00
161,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930,4 days 10:25:00
162,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926,4 days 11:46:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all partners tickets dataset

In [134]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
1,Clarify,7679691,7650034,2021-02-26 17:29:00,2021-02-26 17:29:00,0 days 00:00:00
2,Explain,7679689,7650029,2021-02-26 15:37:00,2021-02-26 15:37:00,0 days 00:00:00
3,Communicate,7679687,7649877,2021-02-26 15:19:00,2021-02-26 15:20:00,0 days 00:01:00
5,Verify,7679685,7649877,2021-02-26 15:19:00,2021-02-26 15:19:00,0 days 00:00:00
9,Clarify,7679681,7649877,2021-02-26 15:18:00,2021-02-26 15:18:00,0 days 00:00:00
...,...,...,...,...,...,...
33815,Communicate,7838831,7826989,2023-10-09 15:07:00,2023-11-13 13:25:00,34 days 22:18:00
33820,Communicate,7709869,7654604,2023-09-27 12:49:00,2023-11-22 15:32:00,56 days 02:43:00
33828,Verify,7709616,7653979,2023-09-19 18:22:00,2023-11-06 09:34:00,47 days 15:12:00
33829,Verify,7709591,7654428,2023-09-19 12:53:00,2023-11-06 14:42:00,48 days 01:49:00


Summarize time for each subtask outside codeubsters

In [135]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,335272,0 days 00:00:00
1,336670,3 days 15:01:00
2,338553,0 days 00:01:00
3,360791,0 days 00:00:00
4,374097,0 days 00:01:00
...,...,...
5250,8048691,0 days 00:01:00
5251,8049403,0 days 02:26:00
5252,8050182,0 days 00:00:00
5253,8055536,0 days 00:00:00


In [136]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_partners` with `all_individual_others_time_sum_df` dataset

In [137]:
all_partners_outside_codebusters_time = pd.merge(all_partners, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_partners_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266,3 days 18:24:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347,1 days 03:59:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078,0 days 10:14:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253,0 days 01:16:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668,0 days 01:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954,0 days 00:17:00
160,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931,0 days 09:14:00
161,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930,0 days 09:18:00
162,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926,0 days 08:49:00


In [138]:
all_partners_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_partners_outside_codebusters_time`, `all_partners_codebusters_time` and `all_partners_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [139]:
all_partners_time_on_and_outside_codebusters = pd.merge(all_partners_outside_codebusters_time, all_partners_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_partners_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266,3 days 18:24:00,6 days 03:05:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347,1 days 03:59:00,12 days 20:14:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078,0 days 10:14:00,6 days 17:43:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253,0 days 01:16:00,6 days 00:26:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668,0 days 01:04:00,0 days 21:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954,0 days 00:17:00,4 days 06:42:00
159,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931,0 days 09:14:00,12 days 13:06:00
160,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930,0 days 09:18:00,4 days 10:25:00
161,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926,0 days 08:49:00,4 days 11:46:00


In [140]:
all_partners_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_partners_review_time` dataset

In [141]:
all_partners_with_time = pd.merge(all_partners_time_on_and_outside_codebusters, all_partners_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_partners_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,9 days 21:29:00,7817266,3 days 18:24:00,6 days 03:05:00,3 days 02:30:00
1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,14 days 00:13:00,7816347,1 days 03:59:00,12 days 20:14:00,8 days 22:53:00
2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,7 days 03:57:00,8018078,0 days 10:14:00,6 days 17:43:00,5 days 20:40:00
3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,6 days 01:42:00,7979253,0 days 01:16:00,6 days 00:26:00,1 days 06:24:00
4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,0 days 22:15:00,7927668,0 days 01:04:00,0 days 21:11:00,0 days 20:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,4 days 06:59:00,7651954,0 days 00:17:00,4 days 06:42:00,4 days 05:14:00
159,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,12 days 22:20:00,7651931,0 days 09:14:00,12 days 13:06:00,10 days 01:50:00
160,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,4 days 19:43:00,7651930,0 days 09:18:00,4 days 10:25:00,3 days 11:15:00
161,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,4 days 20:35:00,7651926,0 days 08:49:00,4 days 11:46:00,3 days 12:32:00


### Merge `all_individual_tasks_communicate_time` with all tikets

In [142]:
all_partners_with_time_communicate = all_partners_with_time.merge(all_individual_tasks_communicate_time, left_on="Issue id", right_on="Parent id", how="left")
all_partners_with_time_communicate.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              1
Resolved-communicate        4
dtype: int64

**Summary**
- there are some null values in `Resolved-communicate`, those are the tickets which do not contain `communicate` step, so propably they wre closed becuse client did not replay or they were not valid

Check for null values

In [143]:
all_partners_with_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              1
dtype: int64

In [144]:
all_partners_with_time.shape

(163, 15)

### Drop rows which contain null value except column `Resolved-communicate`

In [145]:
all_partners_with_time_without_null = all_partners_with_time_communicate.dropna(subset=all_partners_with_time_communicate.columns.difference(['Resolved-communicate']))
all_partners_with_time_without_null.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              0
Resolved-communicate        4
dtype: int64

In [146]:
all_partners_with_time_without_null.shape

(162, 16)

In [147]:
# all_partners_with_time_without_null.to_csv('all_partners.csv')

## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [148]:
all_partners_with_time_converted = all_partners_with_time_without_null
all_partners_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 0 to 162
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   Issue key                 162 non-null    object         
 1   Issue id                  162 non-null    int64          
 2   Status                    162 non-null    object         
 3   Reporter                  162 non-null    object         
 4   Created                   162 non-null    datetime64[ns] 
 5   Due Date                  162 non-null    object         
 6   Resolved                  162 non-null    datetime64[ns] 
 7   Customer                  162 non-null    object         
 8   Type of Request           162 non-null    object         
 9   Is Escalated              162 non-null    bool           
 10  ticket_age                162 non-null    timedelta64[ns]
 11  Parent id                 162 non-null    int64          
 12  time_outside_

Convert `time in..` columns to minutes

In [149]:
all_partners_with_time_converted['time_in_review'] = all_partners_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_partners_with_time_converted['time_on_codebusters'] = all_partners_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_partners_with_time_converted['time_outside_codebusters'] = all_partners_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_partners_with_time_converted['ticket_age'] = all_partners_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_partners_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 0 to 162
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Issue key                 162 non-null    object        
 1   Issue id                  162 non-null    int64         
 2   Status                    162 non-null    object        
 3   Reporter                  162 non-null    object        
 4   Created                   162 non-null    datetime64[ns]
 5   Due Date                  162 non-null    object        
 6   Resolved                  162 non-null    datetime64[ns]
 7   Customer                  162 non-null    object        
 8   Type of Request           162 non-null    object        
 9   Is Escalated              162 non-null    bool          
 10  ticket_age                162 non-null    float64       
 11  Parent id                 162 non-null    int64         
 12  time_outside_codebusters  1

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\842115517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners_with_time_converted['time_in_review'] = all_partners_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_1396\842115517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_partners_with_time_converted['time_on_codebusters'] = all_partners_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\g

In [150]:
all_partners_with_time_converted.reset_index(inplace=True)
all_partners_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate
0,0,CYTCI-10248122,7817266,Closed,alejandro.ruizrobles,2023-10-03 12:41:00,2023-10-10 00:00,2023-10-13 10:10:00,Numiga,Service Request (SR),False,14249.0,7817266,5424.0,8825.0,4470.0,2023-10-12 10:33:00
1,1,CYTCI-10248121,7816347,Closed,alejandro.ruizrobles,2023-10-03 12:06:00,2023-10-10 00:00,2023-10-17 12:19:00,Numiga,Service Request (SR),False,20173.0,7816347,1679.0,18494.0,12893.0,2023-10-17 11:59:00
2,2,CYTCI-9607,8018078,Closed,alejandro.ruizrobles,2023-11-22 12:45:00,2023-11-29 00:00,2023-11-29 16:42:00,Numiga,Service Request (SR),False,10317.0,8018078,614.0,9703.0,8440.0,2023-11-29 15:08:00
3,3,CYTCI-9546,7979253,Closed,alejandro.ruizrobles,2023-11-14 09:17:00,2023-11-21 00:00,2023-11-20 10:59:00,Numiga,Service Request (SR),False,8742.0,7979253,76.0,8666.0,1824.0,2023-11-20 10:39:00
4,4,CYTCI-9495,7927668,Closed,alejandro.ruizrobles,2023-11-02 13:09:00,2023-11-07 00:00,2023-11-03 11:24:00,DERPART,Service Request (SR),False,1335.0,7927668,64.0,1271.0,1240.0,2023-11-03 10:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,158,CYTCI-6523,7651954,Closed,alejandro.ruizrobles,2022-03-03 10:42:00,2022-03-06 00:00,2022-03-07 17:41:00,Numiga,Service Request (SR),False,6179.0,7651954,17.0,6162.0,6074.0,2022-03-07 17:24:00
158,159,CYTCI-6500,7651931,Closed,alejandro.ruizrobles,2022-02-25 14:55:00,2022-03-03 00:00,2022-03-10 13:15:00,Numiga,Service Request (SR),False,18620.0,7651931,554.0,18066.0,14510.0,2022-03-10 10:27:00
159,160,CYTCI-6499,7651930,Closed,alejandro.ruizrobles,2022-02-25 13:51:00,2022-03-03 00:00,2022-03-02 09:34:00,Numiga,Service Request (SR),False,6943.0,7651930,558.0,6385.0,4995.0,2022-03-02 00:16:00
160,161,CYTCI-6495,7651926,Closed,alejandro.ruizrobles,2022-02-25 12:25:00,2022-03-03 00:00,2022-03-02 09:00:00,Numiga,Service Request (SR),False,6995.0,7651926,529.0,6466.0,5072.0,2022-03-02 00:18:00


In [151]:
file_path = Path('../../DataSets/KPIs-final/all_partners_kpi.csv')
all_partners_with_time_converted.to_csv(path_or_buf=file_path)

# Investigation starts

In [152]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 7654264]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
4514,25696,Review,7706959,7654264,2023-06-26 13:21:00,2023-06-28 16:05:00,2 days 02:44:00


In [153]:
all_partners[all_partners["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age


In [154]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,ticket_age,Parent id,time_in_review
108,CYTCI-7876,7653281,Closed,alejandro.ruizrobles,2022-12-15 17:01:00,2022-12-21 00:00,2022-12-23 13:18:00,Numiga,Service Request (SR),False,7 days 20:17:00,NaN,NaT
109,CYTCI-7855,7653260,Closed,alejandro.ruizrobles,2022-12-12 16:54:00,2022-12-15 00:00,2022-12-13 13:52:00,Numiga,Service Request (SR),False,0 days 20:58:00,NaN,NaT
111,CYTCI-7806,7653219,Closed,alejandro.ruizrobles,2022-12-02 16:36:00,2022-12-11 00:00,2022-12-20 10:20:00,Numiga,Service Request (SR),False,17 days 17:44:00,NaN,NaT
112,CYTCI-7796,7653211,Closed,alejandro.ruizrobles,2022-12-01 18:44:00,2022-12-07 00:00,2022-12-12 15:52:00,Numiga,Service Request (SR),False,10 days 21:08:00,NaN,NaT
113,CYTCI-7774,7653189,Closed,alejandro.ruizrobles,2022-11-28 17:19:00,2022-12-13 00:00,2022-12-15 16:36:00,Lufthansa City Center,Service Request (SR),False,16 days 23:17:00,NaN,NaT
114,CYTCI-7773,7653188,Closed,alejandro.ruizrobles,2022-11-28 16:58:00,2022-12-31 00:00,2022-12-20 14:24:00,Numiga,Service Request (SR),False,21 days 21:26:00,NaN,NaT
115,CYTCI-7772,7653187,Closed,alejandro.ruizrobles,2022-11-28 16:24:00,2022-12-12 00:00,2022-12-13 16:59:00,Lufthansa City Center,Service Request (SR),False,15 days 00:35:00,NaN,NaT
116,CYTCI-7756,7653171,Closed,alejandro.ruizrobles,2022-11-23 17:19:00,2022-11-29 00:00,2022-12-12 14:08:00,Numiga,Service Request (SR),False,18 days 20:49:00,NaN,NaT
117,CYTCI-7755,7653170,Closed,alejandro.ruizrobles,2022-11-23 16:38:00,2022-11-29 00:00,2023-01-04 11:46:00,Numiga,Service Request (SR),False,41 days 19:08:00,NaN,NaT
118,CYTCI-7746,7653161,Closed,alejandro.ruizrobles,2022-11-22 11:10:00,2022-11-21 00:00,2022-11-24 17:18:00,Amadeus Pre Sales Demos,Service Request (SR),False,2 days 06:08:00,NaN,NaT


# Investigation ends

# To Do